## Árbol de decisión

Un modelo el cual utiliza varios niveles, donde en cada nivel existe una caractrística que se evalua a través de un umbral, va dividiendo los datos en los que son mayor o menor al umbral definido en la característica.


### Regresión

Este algoritmo usa todas las variables y los splits posibles (umbrales) y elige el umbral en cada variable que reduzca en mayor medida la varianza.


$$
\text{Reducción de varianza} = \text{Varianza total} - \left( \frac{N_1}{N} \times \text{Var}(G1) + \frac{N_2}{N} \times \text{Var}(G2) \right)
$$

### Clasificación

De igual forma que en la regresión, el modelo usa todas las variables y prueba todos los umbrales posibles, eligiendo el que cause una mayor reducción de impureza, la cual se calcula a través del Gini o de la Entriopía.

$$
Gini = 1 - \sum p_i^2
$$

$$
Entropía = -\sum p_i \log_2 p_i
$$


$$
\text{Ganancia de Impureza} = \text{Impureza Nodo Padre} - \left( \frac{N_1}{N} \times \text{Impureza}(G1) + \frac{N_2}{N} \times \text{Impureza}(G2) \right)
$$

---

## Random Forest

Utilizas la técnica de bootsrap para hacer muestreos aleatorios de tus datos, sobre los cuales entrenas tu modelo de árbol de decisión. Cada uno de estos modelos realiza sus predicciones, y el promedio de estas sería la predicción utilizada por el algoritmo de Random Forest.

---

## Gradient Boosting

Proceso iterativo el cual mejora los errores cometidos en pasos anteriores. Inicia con una predicción a la cual le calcula el error, después se entrena un nuevo árbol sobre estos errores, con esto se actualiza la predicción del modelo total. Esto se hace a través de un factor de aprendizaje que pondera las predicciones anteriores y se va repitiendo el proceso.

Cada vez usa el error anterior para en cada iteración ir haciendo esté más pequeño. La predicción del árbol anterior es el promedio de los residuales obtenidos en cada una de las hojas. Modelo nuevo = modelo anteriror + v (nuevo árbol). Para clasificación se utilizan las log-odds.

---

## XGBoost

Sumas árboles óptimos.

XGBoost construye un modelo final a través de una suma de árboles.

$$
\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + f_t(x_i)
$$

Donde:
- $\hat{y}_i^{(t)}$ es la predicción.
- $f_t(x_i)$ es el nuevo árbol.

En cada una de las iteraciones se busca minimizar la contribución marginal a la función de pérdida total.

La función de pérdida se compone de dos partes, una que son los árboles anterirores y otra parte que representa el árbol actual. La parte de los árboles anteriores se usa cmo constante (ya esta dado) y se optimiza lo que aporta el nuevo árbol, llegando a:

$$
\mathcal{L}^{(t)} =
\sum_{i=1}^n l(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)) + \sum_{k=1}^{t} \Omega(f_k)
$$

$\Omega(f_k)$ es un factor de regularización el cual castiga árboles más complejos.

Como la función de pérdida es compleja, se utiliza las series de Taylor para poder aproximar la función de pérdida cerca de de las predicciones. La serie de Taylor de segundo orden permite aproximar la función de pérdida con una forma cuadrática, la cual es más sencilla de optmizar y velve más eficiente el proceso. Utilizando esta expansión, el gradiente y el hessiano la función de pérdida queda de la siguiente forma:

$$
\tilde{\mathcal{L}}^{(t)} = \sum_{i=1}^n \left[ g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2 \right] + \Omega(f_t)
$$

Con ello la optimización es un problema cuadrático más sencillo de optimizar.